******************************************************************************<br>
********* ENTRENAMIENTO CHATBOT BERT EXPRESSITO FULL POWER *******************<br>
******************************************************************************

******************************** IMPORTACIONES *******************************

In [1]:
import tensorflow as tf
import keras as k
import transformers as t

In [2]:
from datasets import load_dataset
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import classification_report, confusion_matrix, auc, roc_curve
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

********************** VARIABLE GLOBAL Y CARGA DE DATOS **************************

Check gpu

In [3]:
tf.config.list_physical_devices('GPU')

2022-01-11 10:16:36.217738: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-11 10:16:36.217784: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gpu): /proc/driver/nvidia/version does not exist


[]

Define batch size

In [4]:
batch_size = 8

Variable global para determinar el path

In [5]:
path = '/home/iexpress/TCGA/data/'

Cargamos los datos

In [6]:
data = pd.read_csv(os.path.join(path_tabular, 'clean_data.csv'), sep = ';')
selected_columns = ["|" not in i for i in tab_dat.columns]
data = tab_dat[tab_dat.columns[selected_columns]]
data_train = data[data.dataset.values == 'train']
data_val = data[data.dataset.values == 'val']
data_test = data[data.dataset.values == 'test']
y_train = data_train['target']
y_val = data_val['target']
y_test = data_test['target']
del data_train['dataset']
del data_val['dataset']
del data_test['dataset']
del data_train['target']
del data_val['target']
del data_test['target']
del data_train['index']
del data_val['index']
del data_test['index']

Using custom data configuration default-d7ba8573d73dfdbb
Reusing dataset csv (/home/iexpress/.cache/huggingface/datasets/csv/default-d7ba8573d73dfdbb/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-bb82920b8a7fea9d
Reusing dataset csv (/home/iexpress/.cache/huggingface/datasets/csv/default-bb82920b8a7fea9d/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-a9ce4f04f913ec8c
Reusing dataset csv (/home/iexpress/.cache/huggingface/datasets/csv/default-a9ce4f04f913ec8c/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

se carga la lista de targets

In [7]:
target_list = pd.read_csv(path + "/topic_levels.csv", sep=";")

 Cargamos el checkpoint de Full Bert

In [8]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(target_list))
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

2022-01-04 22:17:38.714269: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


************************************ TOKENIZACION *********************************

 Creamos una función para tokenizar el dataset

In [9]:
def tokenize_function(data):
    return tokenizer(data["input"], padding="max_length", truncation=True)

 Tratamos los datos del tokenizador

In [10]:
dat_tokenize_train = data_train.map(tokenize_function, batched=True).shuffle(seed=7).remove_columns(["input"]).with_format("tensorflow")
dat_tokenize_val = data_val.map(tokenize_function, batched=True).shuffle(seed=7).remove_columns(["input"]).with_format("tensorflow")
dat_tokenize_test = data_test.map(tokenize_function, batched=True).shuffle(seed=7).remove_columns(["input"]).with_format("tensorflow")

Loading cached processed dataset at /home/iexpress/.cache/huggingface/datasets/csv/default-d7ba8573d73dfdbb/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-ede567f04c0e4009.arrow
Loading cached shuffled indices for dataset at /home/iexpress/.cache/huggingface/datasets/csv/default-d7ba8573d73dfdbb/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-8b2f5c8bb81c9afe.arrow
Loading cached processed dataset at /home/iexpress/.cache/huggingface/datasets/csv/default-bb82920b8a7fea9d/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-f1709c15b2a1eaaa.arrow
Loading cached shuffled indices for dataset at /home/iexpress/.cache/huggingface/datasets/csv/default-bb82920b8a7fea9d/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-ccb4a443dd633f25.arrow
Loading cached processed dataset at /home/iexpress/.cache/huggingface/datasets/csv/default-a9ce4f04f913ec8c/0.0.0/6b9057d9e23d9d8a2f05b985917a0da8

 Obtenemos las features

In [11]:
train_features = {x: dat_tokenize_train["train"][x] for x in tokenizer.model_input_names}
val_features = {x: dat_tokenize_val["train"][x] for x in tokenizer.model_input_names}
test_features = {x: dat_tokenize_test["train"][x] for x in tokenizer.model_input_names}

In [12]:
dat_tf_train = tf.data.Dataset.from_tensor_slices((train_features, dat_tokenize_train["train"]["target"])).batch(batch_size)
dat_tf_val = tf.data.Dataset.from_tensor_slices((val_features, dat_tokenize_val["train"]["target"])).batch(batch_size)
dat_tf_test = tf.data.Dataset.from_tensor_slices((test_features, dat_tokenize_test["train"]["target"])).batch(batch_size)

************************** ENTRENAMIENTO MODELO *********************************

 Usamos Keras para compilar el modelo

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=tf.metrics.SparseCategoricalAccuracy())
#model.layers[0].trainable = False

 Visualizamos su estructura

In [14]:
print(model.summary())

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  177853440 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  13073     
                                                                 
Total params: 177,866,513
Trainable params: 177,866,513
Non-trainable params: 0
_________________________________________________________________
None


Fijamos un callback para generar un early_stopping y guardar el mejor modelo

In [15]:
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]

 Entrenamos el modelo

In [16]:
model_history = model.fit(dat_tf_train, validation_data=dat_tf_test, epochs=10, batch_size=batch_size, callbacks=callbacks)

Epoch 1/10
661/661 [==============================] - 626s 927ms/step - loss: 1.4210 - sparse_categorical_accuracy: 0.5935 - val_loss: 0.8846 - val_sparse_categorical_accuracy: 0.7576
Epoch 2/10
661/661 [==============================] - 612s 926ms/step - loss: 0.5770 - sparse_categorical_accuracy: 0.8441 - val_loss: 0.9667 - val_sparse_categorical_accuracy: 0.7374
Epoch 3/10
661/661 [==============================] - 612s 927ms/step - loss: 0.3682 - sparse_categorical_accuracy: 0.8965 - val_loss: 0.6206 - val_sparse_categorical_accuracy: 0.8249
Epoch 4/10
661/661 [==============================] - 613s 927ms/step - loss: 0.2755 - sparse_categorical_accuracy: 0.9235 - val_loss: 0.5177 - val_sparse_categorical_accuracy: 0.8653
Epoch 5/10
661/661 [==============================] - 613s 928ms/step - loss: 0.2339 - sparse_categorical_accuracy: 0.9366 - val_loss: 0.5012 - val_sparse_categorical_accuracy: 0.8754
Epoch 6/10
661/661 [==============================] - 612s 926ms/step - loss: 0.

 Guardamos el modelo

In [17]:
model.save_pretrained("model_saved_bert")

******************************* VALIDACION *********************************

 Predecimos el set de validación

In [18]:
predictions_test = model.predict(dat_tf_test)

 Multilabel classification

In [19]:
predictions_test_probs = tf.nn.softmax(predictions_test["logits"])

 Obtenemos el valor máximo

In [20]:
predictions_test_classes = np.argmax(predictions_test_probs, 1)

******************************* EVALUACION *********************************

 Evaluamos el modelo(Precision, Recall, F1) 

In [24]:
evaluation_model = classification_report(y_true = dat_tokenize_test["train"]["target"],
                                         y_pred = predictions_test_classes,
                                         target_names = target_list.topic,
                                        output_dict = True)

se guardan las evaluaciones del modelo

In [25]:
evaluaciones_modelo = pd.DataFrame(evaluation_model).transpose()

In [26]:
evaluaciones_modelo.to_csv(path + "/data_evaluation_full_bert.csv")

In [ ]:
evaluaciones_modelo

************************* MATRIZ DE CONFUSION *********************************

In [27]:
  
## Calculamos la matriz de confusión
cm = confusion_matrix(dat_tokenize_test["train"]["target"], predictions_test_classes)
print(cm, "\n")
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm.round(3))

[[13  0  0  1  0  0  1  0  0  1  1  0  0  0  0  0  0]
 [ 0 17  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  1 15  0  1  0  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  1  0 12  0  0  0  0  0  1  1  0  2  0  0  0  0]
 [ 0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0 14  0  0  0  0  1  0  0  0  0  2  0]
 [ 0  1  0  0  0  0 16  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0 14  0  0  1  0  2  0  0  0  0]
 [ 0  1  0  0  0  1  0  0 14  0  0  0  0  0  1  0  0]
 [ 0  1  0  2  0  0  0  0  0 14  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 18  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0 14  2  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  1 15  0  0  0  0]
 [ 1  0  1  0  0  0  0  0  0  0  0  0  0  5  0 11  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0  0  0 16  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0 16  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  6 10]] 

[[0.765 0.    0.    0.059 0.    0.    0.059 0.    0.    0.059 0.059 0.
  0.    

In [28]:
pd.DataFrame(cm).to_csv(path + "/confusion_matrix_full_bert.csv")